# SQLite Databases with Python

* Module Maintainer: @lachlandeer
* Creation Date: August 22, 2017

## Loading the required packages

We need to import the package to interact with SQLite databases, `sqlite3` and we will import `pandas` so that we can get the output as a nicely behaved dataset.

In [1]:
import sqlite3
import pandas as pd

## Connecting to the Auctions Data

The SQLite database is contained in a single file `../database/auctions_data.db`. We connect to it using the function `sqlite3.connect()` which returns a connection object:

In [2]:
connection = sqlite3.connect('../database/auctions_data.db')

In [3]:
type(connection)

sqlite3.Connection

We can then create a Cursor object. Cursors allow us to execute SQL queries against a database:

In [4]:
cursor = connection.cursor()

In [5]:
type(cursor)

sqlite3.Cursor

Now we have the Cursor object, we can use it to execute a query against the database with the aptly named `execute` method. We simply wrap the `SQL` command inside of `.execute()` using quotations, ".

As a first-step we might want to know what tables are in the database. Unlike `R`, we have to do a little work here ourselves:

* The list of tables in a `sqlite` database live inside a master table called `sqlite_master`
* From `sqlite_master` we want to extract the `tables` using a `WHERE` condition

Let's do this:

In [6]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

Note that we don't actually get results from the `execute` method. We need to use the `fetchall()` method applied to our cursor to get the results back:

In [7]:
print(cursor.fetchall())

[('auctions',), ('bidders',), ('bids',)]


which delivers the information we would expect, although in a different format:

In [8]:
results = cursor.fetchall()
type(results)

list

It's good practice to close Connection objects and Cursor objects that are open. This prevents the SQLite database from being locked. When a SQLite database is locked, you may be unable to update the database, and may get errors. We can close the Cursor and the Connection like this:

In [9]:
cursor.close()
connection.close()

## Querying the Database

Now we know what tables are in the database (we already knew! But it's good ot make sure that we got what was expected), we can proceed with making simple queries.

Let's replicate an earlier example where we select AuctionID, Volume, District from the auctions table:

In [10]:
connection = sqlite3.connect('../database/auctions_data.db')
cursor = connection.cursor()

cursor.execute("SELECT AuctionID, Volume, District FROM auctions;")
results = cursor.fetchall()
print(results)

[(1, 1234, 1), (2, 345, 3), (3, 2346, 2), (4, 1278, 4), (5, 789, 7), (6, 934, 6), (7, 269, 9), (8, 357, 8), (9, 1503, 4), (10, 239, 7)]


The results are formatted as a list of tuples. 
Each tuple corresponds to a row in the database. Dealing with data this way is 
painful:

* Need to manually add column heads, and
* Manually parse the data to a csv or dataframe. 

The `pandas` library has an easier way - which will return `DataFrames` to us.

Before we get there, let's be good programmers and remember to close our database connection:

In [11]:
cursor.close()
connection.close()

## Reading SQL results directly into a Pandas DataFrame

`Pandas` has an inbuilt function `read_sql_query` that reads the results of a SQL query straight into a `DataFrame`.

There are several advantages of this:

1. Avoids the need to create a cursor object, and 'fetch' results at the end with `fetch_all`
2. Pandas directly reads in column names from the SQL table headers
3. The output is a `DataFrame` which we have already learned how to work with

Let's take the same query as above, but this time send the results into a pandas `DataFrame`:

In [12]:
connection = sqlite3.connect('../database/auctions_data.db')

data = pd.read_sql_query("SELECT AuctionID, Volume, District FROM auctions;", connection)

data

,AuctionID,Volume,District
0,1,1234,1
1,2,345,3
2,3,2346,2
3,4,1278,4
4,5,789,7
5,6,934,6
6,7,269,9
7,8,357,8
8,9,1503,4
9,10,239,7


Nice! That's almost too easy.

Now suppose we want to execute a more complicated query: again collecting the minimum, maximum and average bid for each bidder, and link this to the bidder’s name.

Writing out the full query in one line would be pretty unreadable; so we want a way to sent a multiline SQL query across and get the results back. We can wrap our query in triple quotations, `"""`, and proceed as usual:

In [13]:
data2 = pd.read_sql_query("""
                            SELECT
                                  bidders.FirstName
                                , bidders.LastName  
                                , MIN(bids.Bid)           AS SmallestBid
                                , ROUND(AVG(bids.Bid),2)  AS AverageBid
                                , MAX(bids.Bid)           AS LargestBid
                            FROM
                                bids
                            INNER JOIN
                                bidders
                            ON
                                bidders.BidderID = bids.BidderID
                            GROUP BY
                                  bids.BidderID
                            ;
                            """,
                             connection)

data2

,FirstName,LastName,SmallestBid,AverageBid,LargestBid
0,Adam,Cooper,10.86,13.53,19.21
1,Bryan,Dykstra,8.81,11.38,13.09
2,Charles,Elan,7.39,10.58,15.62
3,David,Forester,7.93,12.27,15.67
4,Edward,Gulden,7.35,10.09,14.28
5,Frank,Hollister,7.14,9.90,13.03
6,George,Ivanov,7.99,10.13,12.34


We can also write the query as it's own variable and pass that, if we think it makes for tidier code:

In [14]:
request = """
            SELECT
                  bidders.FirstName
                , bidders.LastName  
                , MIN(bids.Bid)           AS SmallestBid
                , ROUND(AVG(bids.Bid),2)  AS AverageBid
                , MAX(bids.Bid)           AS LargestBid
            FROM
                bids
            INNER JOIN
                bidders
            ON
                bidders.BidderID = bids.BidderID
            GROUP BY
                  bids.BidderID
            ;
            """

In [15]:
data3 = pd.read_sql_query(request, connection)
data3

,FirstName,LastName,SmallestBid,AverageBid,LargestBid
0,Adam,Cooper,10.86,13.53,19.21
1,Bryan,Dykstra,8.81,11.38,13.09
2,Charles,Elan,7.39,10.58,15.62
3,David,Forester,7.93,12.27,15.67
4,Edward,Gulden,7.35,10.09,14.28
5,Frank,Hollister,7.14,9.90,13.03
6,George,Ivanov,7.99,10.13,12.34
